# Calculation and Analysis of $\mathcal{G}$

## Load the Package

In [43]:
using ClosedGroupFunctions
ClosedGroupFunctions.ijulia_behavior(:clear)
using Zd_Arithmetics
using SparseArrays

import Zd_Arithmetics: ℤ₂

#push!(LOAD_PATH,"../src/") # for developement, load the local package
using AntiIdClosedGroup

## Generating the group

### Defining the parameters

In [659]:
n = 7 # size of the outer matrix
k = 6 # size of the anti-identity sub-block

6

### The standard generators


In [660]:
Ω, all_pos, all_generators = construct_generators(n,k);

There are Ω = (7 choose 6) = 7  possible basic permutations.


#### Outputting the generators as LaTeX

The generators can be converted to LaTeX-code if needed using

    po2matrix(...)
or

    all_pos2matrix(...)

(See [documentation](https://fhoeddinghaus.github.io/AntiIdClosedGroup.jl/).)

### Generating the groups using the `ClosedGroupFunctions.jl`-Package

In [661]:
group, num_all_multiplications = group_generator_basic(all_generators; prnt=false);

level #1 size 7 and 43 new elements (49 multiplications)
level #2 size 50 and 231 new elements (301 multiplications)
level #3 size 281 and 1015 new elements (1617 multiplications)
level #4 size 1296 and 3430 new elements (7105 multiplications)
level #5 size 4726 and 8379 new elements (24010 multiplications)
level #6 size 13105 and 13083 new elements (58653 multiplications)
level #7 size 26188 and 10408 new elements (91581 multiplications)
level #8 size 36596 and 3409 new elements (72856 multiplications)
level #9 size 40005 and 315 new elements (23863 multiplications)
level #10 size 40320 and 0 new elements (2205 multiplications)

Ended after 10 iterations. 
The resulting group has 40320 elements.


It's not necessary to use the generators, as the `group_generator_basic` takes all kind of elements. Someone may try smaller sets of generators, as seen below (permutations).

It's advised to save the calculated group to disk for later use by using `ClosedGroupFunctions.jl`'s `store_group(...)` and `load_group(...)` functions.

In [ ]:
# store the group to disk
store_group("n$n" * "k$k", group)

In [ ]:
# load the group from disk
group = load_group("n$n" * "k$k");

#### Labelling the group

In [ ]:
# label the generators
all_generators_labelled = label_generators(all_generators);

# label the group with the fast method
#number_of_elements = length(group)
#@time labelled_group = labelled_group_generator_simple(all_generators_labelled, number_of_elements);

# or label the group with the shortest possible label (slower method)
labelled_group = labelled_group_generator_shortest(all_generators_labelled; prnt=false)[1];

In [ ]:
# it's advised to store the labelled group for future use
store_group("n$n" * "k$k", labelled_group; filename_prefix="closed_group_labelled_")

In [662]:
# load the labelled group from disk
labelled_group = load_group("n$n" * "k$k"; filename_prefix="closed_group_labelled_");

## Investigation and Analysis

We now want to investigate the group and it's generators further.

### Permutations

To investigate the group/generators under permutations, we first have to label the elements like above and then calculate all possible permutations.

In [ ]:
all_generators_labelled = label_generators(all_generators);

In [ ]:
# all permutations of [1, 2, 3, 4, 5, 6, 7] can be found with Combinatorics.jl
# either by
# using Combinatorics
# collect(permutations(1:n))
# or without importing (AntiIdClosedGroup depends directly on Combinatorics.jl)
all_permutations = collect(AntiIdClosedGroup.permutations(1:n));

Now we can apply the permutations (or a some of them) to the labelled generators by using the following function. The output is redirected and **appended** into a file with a given prefix (`"permutation_cycles_"`) and the identifier (`"n$n"*"k$k"`), because it's quite long.

In [ ]:
print_permutation_cycles(all_generators_labelled, all_permutations, "n$n" * "k$k")

After some analysis of the file, we find a list of permutations, that can construct the other generators from each other. In the file, `p[i]` corresponds to the `i`th permutation in the entered list, `all_permutations`.

In [ ]:
# in this case, only one permutation is needed to calculate all other generators from one of them.
selected_permutations = [97] # indices of the selected permutations 
ps = all_permutations #[selected_permutations] # use all_permutations to consider the group under all possible permutations

Pₛ = [perm_rows(id(n), ps[i]) for i in 1:length(ps)]
Qₛ = [perm_cols(id(n), ps[i]) for i in 1:length(ps)] # Q = P_inv

# new set of generators
a = all_generators[1]
new_generators = [a, Pₛ..., Qₛ...];

In [ ]:
# calculate the group with the new generators

In [ ]:
group_with_perms, num_all_multiplications_with_perms = group_generator_basic(new_generators; prnt=false, commutes=false);

In [42]:
# As explained in the thesis results, the group stays the same under permutations, 
# because all possible permutations are already part of the group.
# This is therefore a method to possibly find a smaller set of generators.
group == group_with_perms

true

### Invariants and Conjugacy Classes

#### Calculating the conjugacy classes

In [ ]:
conjugacy_classes = calculate_conjugacy_classes(labelled_group);

println("\nThere were $(length(conjugacy_classes)) different conjugacy classes found.")

In [ ]:
store_group("n$n" * "k$k", conjugacy_classes; filename_prefix="conjugacy_classes_")

In [666]:
conjugacy_classes = load_group("n$n" * "k$k"; filename_prefix="conjugacy_classes_");

#### Invariants
The package provides a few invariants that can be used:

1. Order of an element: `ord(g, n)`
2. Trace of an element (with higher powers): `tr_of_power(g, power) = tr(g^power)`
3. Rank of an element (+ Id): `rank_of_power_plus_id(g, power, n) = rank(g^power + id(n))`
4. Generalized Arf invariant of an element: `generalized_arf(g, n; readable = true)`

To use this functions for investigation of the conjugacy classes, we can use the functions `apply_invariant_to_first_in_class(...)` or `apply_invariant_to_first_in_all_classes(...)` from the `ClosedGroupFunctions.jl` package.

Both of these functions expect an `invariant` with exactly one argument: the element `g`. Therefore we have to define wrappers for the invariants, that we want to look at.

In [667]:
prnt = false;

In [668]:
# 1. order
order(g) = ord(g,n)
order_values = apply_invariant_to_first_in_all_classes(conjugacy_classes, labelled_group, order; prnt=prnt);


In [669]:
# 2. trace
# 2.1 power = 1
tr¹(g) = tr_of_power(g, 1)
tr¹_values = apply_invariant_to_first_in_all_classes(conjugacy_classes, labelled_group, tr¹; prnt=prnt);
println("")

# 2.2 power = 2
tr²(g) = tr_of_power(g, 2)
tr²_values = apply_invariant_to_first_in_all_classes(conjugacy_classes, labelled_group, tr²; prnt=prnt);

# ... maybe higher powers

In [670]:
# 3. rank
# 3.1 rank(g^1 + I)
rank¹(g) = rank_of_power_plus_id(Matrix(g), 1, n) # sometimes a wrapping of g with Matrix(g) is needed
rank¹_values = apply_invariant_to_first_in_all_classes(conjugacy_classes, labelled_group, rank¹; prnt=prnt);
println("")

# 3.2 rank(g^2 + I)
rank²(g) = rank_of_power_plus_id(Matrix(g), 2, n)
rank²_values = apply_invariant_to_first_in_all_classes(conjugacy_classes, labelled_group, rank²; prnt=prnt);

# ... maybe higher powers

In [671]:
# 4. GArf
garf(g) = generalized_arf(g, n; readable=true)
garf_values = apply_invariant_to_first_in_all_classes(conjugacy_classes, labelled_group, garf; prnt=prnt);


#### Pretty output of the values using a HTML table

In [672]:
# list of invariants (columns) to print out in the form of label => values
column_keys = ["# of elements in Cl(x)", "ord(g)", "tr(g^1)", "tr(g^2)", "rank(g^1 + 1)", "rank(g^2 + 1)", "GArf [radian, °]"]
columns = Dict([
        "# of elements in Cl(x)" => length.(conjugacy_classes),
        "ord(g)" => order_values,
        "tr(g^1)" => tr¹_values,
        "tr(g^2)" => tr²_values,
        "rank(g^1 + 1)" => rank¹_values,
        "rank(g^2 + 1)" => rank²_values,
        "GArf [radian, °]" => garf_values
        ])

HTML() do io
    println(io, "<table><thead>
            <tr>
                <td>Conjugacy Class No.</td>
                <td>x for Cl(x)</td>")
    for l in column_keys
        println(io, "<td>" * l * "</td>")
    end
    println(io, "
                
            </tr>
        </thead><tbody>")
    for i in 1:length(conjugacy_classes)
        println(io, "<tr><td>$i</td><td>" * sort(collect(conjugacy_classes[i]))[1] * "</td>")
        for l in column_keys
             println(io, "<td>$(columns[l][i])</td>")
        end
        println(io, "</tr>")
    end
    
    println(io, "</tbody></table>")
end

Conjugacy Class No.,x for Cl(x),# of elements in Cl(x),ord(g),tr(g^1),tr(g^2),rank(g^1 + 1),rank(g^2 + 1),"GArf [radian, °]"
1,a,28,2,1,1,1,0,"(""0.0π"", ""0.0°"")"
2,aa,1,1,1,1,0,0,"(""-0.25π"", ""-45.0°"")"
3,ab,112,3,0,0,3,3,"(""0.75π"", ""135.0°"")"
4,abacd,1120,6,0,0,4,3,"(""0.0π"", ""0.0°"")"
5,abadac,1344,5,0,0,5,5,"(""0.75π"", ""135.0°"")"
6,abadcd,210,2,1,1,2,0,"(""0.0π"", ""0.0°"")"
7,abadce,2520,4,1,1,4,2,"(""0.0π"", ""0.0°"")"
8,abaecad,3360,6,1,1,5,6,"(""0.0π"", ""0.0°"")"
9,abaedcac,1120,3,1,1,6,6,"(""-0.25π"", ""-45.0°"")"
10,abaedcd,420,2,1,1,3,0,"(""0.0π"", ""0.0°"")"
